In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
#Data preprocessing
childcare = pd.read_csv(r'project1_new_datasets\new_child_care.csv')
employment = pd.read_csv(r'project1_new_datasets\new_employment.csv')
income = pd.read_csv(r'project1_new_datasets\new_income.csv')
population = pd.read_csv(r'project1_new_datasets\new_population.csv')
potential_loc = pd.read_csv(r'project1_new_datasets\new_potential_loc.csv')


population = population.iloc[:, :5].drop(['Total'], axis=1)
population['2w-12yrs'] = np.floor(population.iloc[:, 1:].sum(axis=1)*13/15).astype(int)
demand_desert = pd.merge(population, employment, on='zip_code', how = 'outer')
demand_desert = pd.merge(demand_desert, income, on='zip_code', how = 'outer')
demand_desert['high_demand'] = (demand_desert['employment rate'] >= 0.6)|(demand_desert['average income'] <= 60000)
demand_desert['high_demand'] = demand_desert['high_demand'].astype(int)
#Data cleaning
childcare=childcare[childcare['total_capacity']>0].reset_index(drop=True)
childcare_capacity = childcare.groupby('zip_code')[['infant_capacity','toddler_capacity','preschool_capacity','school_age_capacity'
                                                   ,'children_capacity']].sum().reset_index()
childcare_capacity['2w_5yr_cap'] = np.floor(childcare_capacity.iloc[:, 1:4].sum(axis=1)+childcare_capacity['children_capacity']*5/12).astype(int)
childcare_capacity['2w_12yr_cap'] = np.floor(childcare_capacity.iloc[:, 1:6].sum(axis=1)).astype(int)

demand_desert = pd.merge(demand_desert, childcare_capacity, on='zip_code', how = 'outer')

#def classify_desert(row):
#    if row['high_demand'] == 1:
#        return row['2w_12yr_cap'] <= row['2w-12yrs']*0.5
#    else:
#        return row['2w_12yr_cap'] <= row['2w-12yrs']*1/3
#
#demand_desert['desert'] = demand_desert.apply(classify_desert, axis=1).astype(int)

# demand_desert.to_csv(r'project1_new_datasets\demand_desert.csv', index=False)



In [2]:
#Problem 1
m1=Model("Budgeting")
x={}
y={}
#Decision variables
for i in range(len(childcare)):
    x[1,i]=m1.addVar(vtype=GRB.INTEGER,name=f"new general slots at facility {i}")
    x[2,i]=m1.addVar(vtype=GRB.INTEGER,name=f"new slots for children under 5 at facility {i}")
for j in range(len(demand_desert)):
    y[1,j]=m1.addVar(vtype=GRB.INTEGER,name=f"newly built small facilities in {j}")
    y[2,j]=m1.addVar(vtype=GRB.INTEGER,name=f"newly built medium facilities in {j}")
    y[3,j]=m1.addVar(vtype=GRB.INTEGER,name=f"newly built large facilities in {j}")
#Objective function
m1.setObjective(
    quicksum(65000*y[1,j]+95000*y[2,j]+115000*y[3,j] for j in range(len(demand_desert)))+
        quicksum(200*x[1,i]+300*x[2,i]+20000*((x[1,i]+x[2,i])/childcare["total_capacity"][i])
            for i in range(len(childcare))),GRB.MINIMIZE
)
#Constraints
for i in range(len(childcare)):
    m1.addConstr(x[1,i]+x[2,i]<=0.2*childcare["total_capacity"][i],f"Maximum expansion rate {i}")
    m1.addConstr(childcare["total_capacity"][i]+x[1,i]+x[2,i]<=max(childcare["total_capacity"][i],500),f"Maximum slots {i}")
    m1.addConstr(x[1,i]>=0,f"non-negativity {i}_1")
    m1.addConstr(x[2,i]>=0,f"non-negativity {i}_2")
    
care_reg=childcare.groupby("zip_code")
for j in range(len(demand_desert)):
    m1.addConstr(y[1,j]>=0,f"non-negativity {j}_y_1")
    m1.addConstr(y[2,j]>=0,f"non-negativity {j}_y_2")
    m1.addConstr(y[3,j]>=0,f"non-negativity {j}_y_3")
    jcare=care_reg.get_group(demand_desert.iloc[j,0]).index
    #high demand or not(changing greater than to greater than or equal)
    if demand_desert["high_demand"][j]==1:
        m1.addConstr(demand_desert["2w_12yr_cap"][j]+100*y[1,j]+200*y[2,j]+400*y[3,j]+
                         quicksum(x[1,l]+x[2,l] for l in jcare)>=int(1/2*(demand_desert["2w-12yrs"][j]))+1)
    else:
        m1.addConstr(demand_desert["2w_12yr_cap"][j]+100*y[1,j]+200*y[2,j]+400*y[3,j]+
                         quicksum(x[1,l]+x[2,l] for l in jcare)>=int(1/3*(demand_desert["2w-12yrs"][j]))+1)
    m1.addConstr(demand_desert["2w_5yr_cap"][j]+50*y[1,j]+100*y[2,j]+200*y[3,j]+
                         quicksum(x[2,l] for l in jcare)>=int(2/3*(demand_desert["-5"][j]))+1)
m1.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-09
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 64135 rows, 32579 columns and 142002 nonzeros
Model fingerprint: 0xf1e59749
Variable types: 0 continuous, 32579 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [2e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 1e+04]
Found heuristic solution: objective 3.611000e+08
Presolve removed 61699 rows and 23307 columns
Presolve time: 0.13s
Presolved: 2436 rows, 9272 columns, 14275 nonzeros
Found heuristic solution: objective 3.414887e+08
Variable types: 0 continuous, 9272 integer (1003 binary)
Found heuristic solution: objective 3.165964e+08

Root relaxation: objectiv

In [3]:
if m1.status == GRB.OPTIMAL:
    for i in range(len(childcare)):
        print(f"Expand facility {i} by {x[1,i].x} slots for all and {x[2,i].x} slots for children under the age of 5")
    for j in range(len(demand_desert)):
        print(f"Build {y[1,j].x} small,{y[2,j].x} medium, {y[3,j].x} large facilities in region {demand_desert['zip_code'][j]}")
else:
    print("No optimal solution found")

Expand facility 0 by 0.0 slots for all and 12.0 slots for children under the age of 5
Expand facility 1 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 2 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 3 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 4 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 5 by 0.0 slots for all and 17.0 slots for children under the age of 5
Expand facility 6 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 7 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 8 by 0.0 slots for all and 58.0 slots for children under the age of 5
Expand facility 9 by 0.0 slots for all and 0.0 slots for children under the age of 5
Expand facility 10 by 0.0 slots for all and 61.0 slots for children under the age of 5
Expand facility 11 by 0.0 slots for all and 28.0 slots for c

In [16]:
from geopy import distance
distance.distance((childcare["latitude"][0],childcare["longitude"][0]),(childcare["latitude"][1],childcare["longitude"][1])).miles

0.09124712427874171